In [1]:
%load_ext autoreload
%autoreload 2

In [92]:
from pathlib import Path

import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from energizer.datastores import PandasDataStoreForSequenceClassification
from src.utilities import downsample_positive_class
from logging import getLogger
from pprint import pprint
logger = getLogger("log")

In [4]:
ds_dict = load_dataset("pietrolesci/wiki_toxic_indexed")
ds_dict = downsample_positive_class(
    dataset_dict=ds_dict,
    target_name="labels",
    positive_class=1,
    proportion=0.05,
    seed=42,
    logger=logger,
)

Found cached dataset parquet (/home/pl487/.cache/huggingface/datasets/pietrolesci___parquet/pietrolesci--wiki_toxic_indexed-bbeb1b8d65bf4665/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
ds = PandasDataStoreForSequenceClassification()
ds.from_dataset_dict(
    ds_dict,
    input_names=["text"],
    target_name="labels",
    uid_name="uid",
)

In [8]:
emb_name = "embedding_all-mpnet-base-v2"
ds.add_index(emb_name)

In [9]:
pos = ds.data.loc[ds.data["labels"] == 1]

In [16]:
pos

,id,comment_text,labels,uid,embedding_all-mpnet-base-v2,embedding_multi-qa-mpnet-base-dot-v1,embedding_all-MiniLM-L12-v2,is_labelled,is_validation,labelling_round
12,30dc274513233575,FUCK FUCKITY FUCK FUCK FUCK,1,12,"[-0.053848498, -0.0074524563, -0.0035066863, -...","[-0.3526661, -0.23270306, -0.44890895, -0.1981...","[-0.070474654, 0.084659934, -0.047556177, -0.0...",False,False,-100
22,a2f59e64d8d5c396,And I don't understand what you try to tell me...,1,24,"[-0.018567849, 0.12365515, -0.018676337, 0.000...","[-0.15598983, 0.040226553, -0.44666332, -0.113...","[0.068033814, 0.009724658, 0.034210224, -0.011...",False,False,-100
53,8003c3ecdd9a8509,"Wikipedia:Don't lie, which you clearly are. Di...",1,59,"[0.009366356, 0.043653578, -0.020597767, 0.025...","[-0.08701086, -0.007008126, -0.20551905, -0.45...","[0.0356024, 0.046609286, 0.0045743543, 0.04153...",False,False,-100
95,4eb02da96c1a4640,What a bunch of wanker's,1,102,"[-0.01656083, 0.005105335, 0.01302901, -0.0279...","[-0.123250306, -0.36482674, -0.4108042, 0.1177...","[-0.02953031, -0.00042376353, 0.03897908, -0.0...",False,False,-100
123,d01127de6693c2f7,""":::Well i see you think that u are infact bet...",1,131,"[0.034338906, -0.028612122, 0.01829433, 0.0474...","[0.25865602, -0.3579902, -0.2437779, 0.1697282...","[-0.0032080766, 0.051220685, 0.055850286, -0.0...",False,False,-100
...,...,...,...,...,...,...,...,...,...,...
152943,67fde260ec36809d,"""\n\n BREAKING NEWS!!!! I KNOW THIS IS ONLY F...",1,159495,"[0.025847761, 0.038410705, 0.01219145, 0.03788...","[0.2613128, -0.1784562, -0.12551399, -0.095510...","[0.00516866, -0.06546281, -0.007649498, -0.007...",False,False,-100
152946,eaa0628fcf7f6dd7,"""\n\n Ryanpostlethwaite \n\nI'm not going to s...",1,159498,"[0.10523507, 0.060824562, -0.0010460714, 0.008...","[0.682058, -0.01012851, -0.22614306, -0.218176...","[0.028509254, -0.026426306, 0.036981452, -0.03...",False,False,-100
152987,0d40571afbaa33b7,sockpuppet \n\nwhose sockpuppet are you?,1,159539,"[0.020607375, 0.0832849, 0.009334755, -0.00023...","[0.0017106512, -0.064978406, -0.2753086, -0.16...","[-0.110520415, -0.04415236, 0.07223461, -0.053...",False,False,-100
153004,ae669be571fb0a81,I don't give a flying fuck. block me I Don't c...,1,159556,"[-0.007177052, -0.015342679, -0.0263921, 0.022...","[0.25146964, -0.53583264, -0.4261305, -0.21381...","[-0.014942829, 0.024863917, 0.00075316406, -0....",False,False,-100


In [52]:
%%time
v, d = ds.search(np.stack(pos[emb_name].values), query_size=100)


CPU times: user 30.4 s, sys: 2.3 ms, total: 30.4 s
Wall time: 4.4 s


In [87]:
v_flat = np.unique(v[:, :10].flatten()).tolist()
ds.get_by_ids(v_flat)["labels"].value_counts()

labels
0    30001
1     7310
Name: count, dtype: int64

In [99]:
v.shape

(9673, 100)

In [97]:
df = ds.get_by_ids(v_flat)
for t in df.loc[df["labels"] == 0, "comment_text"].sample(30).tolist():
    pprint(f"{t}\n========")

('Delete my page: Rajeev Jain \n'
 '\n'
 'Dear Timtrent,\n'
 '\n'
 'My name is Rajeev Jain and I am a Director of Photography by profession. I '
 'have yesterday/ today been appraise by my friend Aakash Pande '
 '(http://en.wikipedia.org/wiki/Aakash_Pandey) a page about me '
 '(http://en.wikipedia.org/wiki/Draft:Rajeev_Jain) on Wikipedia by a fake '
 "author 'Jeff' 'Bhartendunatyaakademi'. I would like to share that this page "
 '(http://en.wikipedia.org/wiki/Draft:Rajeev_Jain) is hugely distressing to me '
 'and I request you to kindly delete this page at the earliest. My web page '
 "'www.rajeevjain.com' provides information about myself for your reference. \n"
 '\n'
 'I look forward to your urgent attention to this matter.\n'
 '\n'
 'Yours faithfully,\n'
 'Rajeev Jain\n'
 '+ 91 982 007 3366\n'
 '+ 91 983 396 0766\n'
 'rajeev@rajeevjain.com\n'
 'www.rajeevjain.com\n'
 '========')
('"\n'
 '\n'
 ' Private info on image page \n'
 '\n'
 'This is probably not a problem. The e-mail explain